# Quantum Teleportation

Quantum teleportation is one of the marquee phenomena of quantum computing. While the name conjures sci-fi fantasy, 
quantum teleportation is an experimentally realised [ ] in qubits [1] and more recently qutrits [2]. In this notebook we follow and add to the code from Jack Hidary's *Quantum Computing: An Applied Approach* (website can be found: https://github.com/JackHidary/quantumcomputingbook) [3].

We begin by importing the relevant packages and gates with a check on the version of cirq being used.

In [92]:
import cirq
import random
from cirq.ops import H, X, I, CNOT, Y, measure

print("Cirq version:", cirq.__version__)

Cirq version: 0.6.0


## Foundational elements

When first using cirq, you will notice that the qubit will always start in the 0 state. Let's check this out!

In [93]:
# Initialize the circuit and the simulator
circuit = cirq.Circuit()
sim = cirq.Simulator()

# Define the number of qubits. In cirq, we can either use
#     a 'GridQubit' or a 'LineQubit' for simplicity in this instance
#     we will use a 'LineQubit'
qubit = cirq.LineQubit(1)

# Append an identity gate and then a measure gate
circuit.append(I(qubit))
circuit.append(measure(qubit))

# Simulate the circuit and print the measurement result
final_results = sim.simulate(circuit)
print(final_results)

# Print the Bloch Sphere coordinates. Cirq has a function that
#    returns the coordinates as a vector. The first argument is 
#    the state you want to check, and the second argument is the 
#    index of the qubit you want to show.
print("\nBloch Sphere coordinates of a 'default' qubit:")
b0X, b0Y, b0Z = cirq.bloch_vector_from_state_vector(final_results.final_state, 0)

print("x: ", round(b0X, 4),
      "\ny: ", round(b0Y, 4),
      "\nz: ", round(b0Z, 4))

measurements: 1=0
output vector: |0⟩

Bloch Sphere coordinates of a 'default' qubit:
x:  0.0 
y:  0.0 
z:  1.0


The 0 state is not exactly the most fun qubit to teleport. So, how do we make an arbitrary, interesting qubit to teleport? We pass our qubit through the X gate raised to a random exponent and the Y gate raised to a random exponent. This will bring our 0-ket to an arbitrary state on the bloch sphere.

We will do this now.

In [94]:
# Initialize the circuit and the simulator
circuit = cirq.Circuit()
sim = cirq.Simulator()

# Choose random exponents for our X-gate and Y-gate
ranX = random.random()
ranY = random.random()

# Define the number of qubits
qubit = cirq.LineQubit(1)

# Append an X-gate and a Y-gate raised to the random exponents
circuit.append([cirq.X(qubit)**ranX,cirq.Y(qubit)**ranY])

# Simulate the circuit
final_results = sim.simulate(circuit)

# Print the Bloch Sphere coordinates. Cirq has a function that
#    returns the coordinates as a vector. The first argument is 
#    the state you want to check, and the second argument is the 
#    index of the qubit you want to show.
print("Bloch Sphere coordinates of 'random' qubit:")
b0X, b0Y, b0Z = cirq.bloch_vector_from_state_vector(final_results.final_state, 0)

print("x: ", round(b0X, 4),
      "\ny: ", round(b0Y, 4),
      "\nz: ", round(b0Z, 4))

Bloch Sphere coordinates of 'random' qubit:
x:  -0.6099 
y:  -0.7903 
z:  0.0589


This is much more interesting! Now, we will begin to construct the teleportation circuit. We will use the circuit shown in [1].

## Teleportation circuit

As is typical in the documentation, we will construct the circuit by defining a function.

In [95]:
def make_quantum_teleportation_circuit():
    '''Builds the teleportation circuit for a random
    qubit to be sent from Alice to Bob'''
    
    # Define the random powers for the qubit to be
    #     sent
    ranX = random.random()
    ranY = random.random()
    
    # Initialise the circuit and the simulator
    circuit = cirq.Circuit()
    sim = cirq.Simulator()
    
    # Prepare the qubits to be used. Alice has 'alice' 
    #     and 'msg,' which will be the random qubit to 
    #     be sent to Bob, who has 'bob'
    msg, alice, bob = cirq.LineQubit.range(3)
        
    # Check what the message is to ensure the teleportation
    #     is successful
    message = sim.simulate(cirq.Circuit([X(msg)**ranX,Y(msg)**ranY]))
    print("Bloch Sphere coordinates of Alice's qubit:")
    aX, aY, aZ = cirq. bloch_vector_from_state_vector(message.final_state, 0)
    print("x: ", round(aX, 4),
          "\ny: ", round(aY, 4),
          "\nz: ", round(aZ, 4))
    
    # Create the random qubit to be teleported
    circuit.append([X(msg)**ranX, Y(msg)**ranY])
    
    # Entangle Alice's non-message qubit with Bob's
    #     qubit
    circuit.append([H(alice),CNOT(alice, bob)])
    
    # Instantiate the circuit to teleport the message
    circuit.append([CNOT(msg, alice), H(msg)])
    circuit.append(measure(msg, alice))
    
    # Manipulate Bob's qubit in order to successfully
    #     send the message based on the syndrome from
    #     Alice's measurements.
    circuit.append([CNOT(alice, bob),CZ(msg, bob)])
    
    # Print out the circuit so that we can check it is 
    #     correctly instantiated
    print("\nCircuit:\n")
    print(circuit)
    
    # Simulate the circuit to teleport the qubit
    final_results = sim.simulate(circuit)
    
    # Print the results of Bob's qubit
    print("\nBloch Sphere coordinates of Bob's qubit:")
    bX, bY, bZ = cirq.bloch_vector_from_state_vector(final_results.final_state, 2)
    print("x: ", round(bX, 4),
          "\ny: ", round(bY, 4),
          "\nz: ", round(bZ, 4))
    
    # Check if it is successfully teleported
    if round(aX, 4) == round(bX, 4) and round(aY, 4) == round(bY, 4) and round(aZ, 4) == round(bZ, 4):
        print("\nTeleportation successful!")
    else:
        print("\nTeleportation unsuccessful :(")

In [96]:
make_quantum_teleportation_circuit()

Bloch Sphere coordinates of Alice's qubit:
x:  -0.5798 
y:  -0.5569 
z:  -0.5947

Circuit:

0: ───X^(13/16)───Y^0.246───@───H───M───────@───
                            │       │       │
1: ───H───────────@─────────X───────M───@───┼───
                  │                     │   │
2: ───────────────X─────────────────────X───@───

Bloch Sphere coordinates of Bob's qubit:
x:  -0.5798 
y:  -0.5569 
z:  -0.5947

Teleportation successful!


### References

[1] Bouwmeester, D.; et al. (1997). "Experimental quantum teleportation". Nature. 390 (6660): 575–579. arXiv:1901.11004.

[2] Luo, Y. -han,; et al, (2019). "Quantum Teleportation in High Dimensions", Physical Review Letters. 

[3] Hidary, J., (2019). *Quantum Computing: An Applied Approach*. Cham: Springer.